# k-Fold CV

- データをk個に分割して交差検証をする方法
- よく使われる手法
- LOOCVよりもコストがかからない　
- LOOCVと違ってランダム性がある

In [22]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = sns.load_dataset('tips')
X = df['total_bill'].values.reshape(-1, 1)
y = df['tip']

In [12]:
k = 5
cv = KFold(n_splits=k, shuffle=True, random_state=0)
model = LinearRegression()
mse_list = []

In [11]:
list(cv.split(X))

[(array([  0,   1,   2,   3,   4,   6,   7,   9,  10,  11,  13,  14,  16,
          17,  19,  20,  21,  23,  24,  25,  26,  27,  28,  29,  30,  31,
          32,  33,  34,  35,  36,  38,  39,  40,  41,  42,  43,  46,  47,
          48,  49,  50,  51,  52,  53,  54,  56,  57,  58,  59,  60,  61,
          62,  65,  66,  67,  68,  69,  70,  72,  75,  77,  78,  79,  80,
          81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  93,  94,
          95,  96,  97,  98,  99, 100, 101, 102, 103, 105, 106, 109, 112,
         113, 114, 115, 116, 117, 119, 120, 121, 122, 123, 126, 127, 128,
         129, 130, 131, 132, 133, 134, 135, 137, 139, 140, 141, 142, 143,
         144, 146, 147, 148, 149, 151, 152, 153, 154, 156, 157, 159, 160,
         161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174,
         175, 176, 177, 178, 179, 182, 183, 184, 185, 186, 187, 190, 191,
         192, 193, 194, 195, 196, 197, 200, 201, 202, 204, 205, 206, 207,
         208, 211, 213, 214, 215, 216,

In [13]:
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # モデル学習
    model.fit(X_train, y_train)
    # テストデータの予測
    y_pred = model.predict(X_test)
    # MSEの計算
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)   

In [16]:
print(f'MSE({k}FoleCV): {np.mean(mse_list)}')
print(f'std({k}FoldCV):{np.std(mse_list)}')

MSE(5FoleCV): 1.0802110883943918
std(5FoldCV):0.1617010050703952


# cross_val_scoreを使う場合

In [19]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)

In [21]:
print(f'MSE({k}FoleCV): {-np.mean(scores)}')
print(f'std({k}FoldCV):{np.std(scores)}')

MSE(5FoleCV): 1.080211088394392
std(5FoldCV):0.1617010050703952


# Repeated k-Fold CV

- ランダム性を抑えるためにk-Fold CVを複数回実施

In [25]:
k = 5
n_repeats = 3
cv = RepeatedKFold(n_splits=k, n_repeats=n_repeats, random_state=0)
model = LinearRegression()
mse_list = []

In [26]:
for train_index, test_index in cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # モデル学習
    model.fit(X_train, y_train)
    # テストデータの予測
    y_pred = model.predict(X_test)
    # MSEの計算
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)   

In [27]:
mse_list

[0.8213090642766285,
 1.0745842125927971,
 1.0880123892600388,
 1.3323867714930204,
 1.084763004349474,
 1.1587839131131425,
 1.6042084002514578,
 1.0307086207441927,
 0.7120290668798744,
 0.8472985410140895,
 0.8856103319481907,
 1.5248521639391936,
 0.6332659028150582,
 1.2003542002626073,
 1.121414266809207]

In [28]:
scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)

In [29]:
scores

array([-0.82130906, -1.07458421, -1.08801239, -1.33238677, -1.084763  ,
       -1.15878391, -1.6042084 , -1.03070862, -0.71202907, -0.84729854,
       -0.88561033, -1.52485216, -0.6332659 , -1.2003542 , -1.12141427])